## Preprocess Metacritic, Genres, and Supported Platforms Columns

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('../../data/raw/info_base_games.csv', low_memory=False)

In [ ]:
df.shape

Checking null values

In [ ]:
df.isna().sum().sort_values(ascending = False)

### Preprocessing the Metacritic Score Column

[Metacritic](https://www.metacritic.com/) is a website for critics and users to review digital content (movies, music, **games**, etc.), the metacritic score that is included on Steam is the **critics** (professional game reviewers) metacritic score, not the users.

In [ ]:
# Converting the metacritic column to numeric since it contains NaNs
df['metacritic'] = pd.to_numeric(df['metacritic'], errors='coerce')

Checking Percentage of Missing Values

In [ ]:
total = len(df)
non_null = df['metacritic'].notna().sum()
missing = df['metacritic'].isna().sum()
print(f"Total rows: {total}")
print(f"With metacritic score: {non_null} ({non_null/total:.1%})")
print(f"Missing metacritic score: {missing} ({missing/total:.1%})")

The above results show that almost 97% of the games in our data don't have a metacritic score associated with them and the metacritic score for them is null, this is due to Steam leaving it **optional** for games' publishers to include a metacritic score on their Steam page.

In [ ]:
df['metacritic'].describe()

In [ ]:
plt.hist(df['metacritic'].dropna(), bins=20)
plt.xlabel("Metacritic score")
plt.ylabel("Number of games")
plt.title("Distribution of Metacritic Scores Ignoring Null Values")
plt.show()

The above histogram shows that the **metacritic scores are normally distributed**, which means **we can apply standardization on it**.

Metacritic scores before standardization

In [ ]:
df['metacritic'].dropna().head()

Standardizing metacritic scores

In [ ]:
standardizer = StandardScaler()
# IMPORTANT: later on I should fit_transform only on the training data
df['metacritic_preprocessed'] = standardizer.fit_transform(df[['metacritic']])

df['metacritic_preprocessed'].dropna().head()

Now a problem remains, which is the missing values, it doesn't make sense to drop all the rows that don't contain a metacritic score since 97% of the data doesn't have it, the best solution that came to my mind is to **set all the NaN metacritic scores** to be equal to the **mean of the standardized metacritic scores**, which is **0**. And create a new boolean feature `has_metacritic` which indicates whether this game has a metacritic score or not, I hope that this can help models understand that if `has_metacritic = 0` then ignore this metacritic score, and also generally `has_metacritic` may later on turn out to be a useful feature on its own.

Creating New `has_metacritic` Feature

In [ ]:
df['has_metacritic'] = df['metacritic'].notna().astype(int)
print(df['has_metacritic'].head())
print(df['has_metacritic'].loc[[1893]])

#### Replacing Missing Values With the Mean Value

Before

In [ ]:
df['metacritic_preprocessed'].head()

After

In [ ]:
df['metacritic_preprocessed'] = df['metacritic_preprocessed'].fillna(0)
print(df['metacritic_preprocessed'].head())

In [ ]:
df.head()

#### Final Thoughts On Preprocessing Metacritic

The above concludes my current trials to preprocess the metacritic column. I will leave below some thoughts that we may wish to revisit in the future:
1. I am not sure if the imputation technique that I used is the most suitable technique for this case, and I am not sure if it is correct to standardize then impute, or should I impute first then standardize, I chose the first approach since I think this will better maintain the distribution of the original data, and especially since I believe the imputed value shouldn't have any more meaning than just indicating that this row didn't have a metacritic value, which I tried to do along with the `has_metacritic` column.

2. Most importantly, I believe later on the `metacritic` column **will turn out to not be useful** for our models and that we will remove it as a feature, this is based on some discussions I read ([discussion1](https://Steamcommunity.com/discussions/forum/10/3057367211653181335/?l=latam), [discussion2](https://www.reddit.com/r/pcgaming/comments/1gjadpf/da_tv_metacritic_user_score_vs_Steam_ratings/)) in which multiple people feel that it doesn't give accurate reviews, I also expect that it might have a high correlation with `reviewScore` in `gamalytic_Steam_games.csv`, and considering that a small number of samples have this score, I expect this feature won't be useful, on the other hand, I believe we might find the new `has_metacritic` feature useful on its own, still this is all just speculations and we will find out by using proper feature selection methods.